In [1]:
# Import URLValidator
from url_validator import URLValidator

# Import SerpAPI key from local python file api_key.py, comment out if manually putting in a key
from api_keys import serpapi_key

/home/josh/.local/lib/python3.8/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2025-02-17 05:30:06.421442: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-17 05:30:06.460151: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-17 05:30:07.316271: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
api_key = serpapi_key

validator = URLValidator(api_key)

In [3]:
user_prompt = "How do economic indicators influence global stock market trends?"
url_to_check = "https://www.investopedia.com/articles/investing/031413/economic-indicatiors-affect-us-stock-market.asp"
citation = False
knowledge_domain = "Finance"

validator.rate_url_validity(user_prompt, url_to_check, flags = {'citation': citation, 'domain': knowledge_domain})

{'raw_scores': {'content_relevance': 4.17,
  'domain_trust': 4.95,
  'title_relevance': 1.67,
  'final_score': 4.57},
 'stars': {'score': 4.57, 'icon': '⭐⭐⭐⭐🌟'},
 'explanation': 'The webpage content is highly relevant to the query. The webpage domain is highly trusted, or the webpage links to many credible domains. The webpage url is not related to the query.'}